# Verify srt parse

In [ ]:
from google.colab import drive, userdata
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import re
from google.colab import files

# === SRT parsing pattern ===
pattern = r"(\d+)\s+([\d:,]+ --> [\d:,]+)\s+(.+?)(?=\n\d+\n|$)"

# Function to process and verify SRT content
def verify_srt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        srt_content = f.read()

    # Extract subtitle entries using regex
    entries = re.findall(pattern, srt_content, flags=re.DOTALL)

    # === Verification Loop ===
    for entry in entries:
        num = entry[0]  # Subtitle number
        timestamp = entry[1]  # Timestamp
        text = entry[2].strip()  # Subtitle text

        # Print the details of each entry
        print(f"Segment {num} - Timestamp: {timestamp}")
        print(f"   Selected Text: '{text}'")
        print("-" * 40)

# Step 1: Upload SRT file in Colab
uploaded = files.upload()

# Step 2: Get the uploaded file name
uploaded_file_name = list(uploaded.keys())[0]

# Step 3: Verify the uploaded file content
print(f"Processing the uploaded file: {uploaded_file_name}\n")
verify_srt_file(uploaded_file_name)


Saving Copy of Chapter 1A - Introduction to DTP (2).srt to Copy of Chapter 1A - Introduction to DTP (2).srt
Processing the uploaded file: Copy of Chapter 1A - Introduction to DTP (2).srt

Segment 1 - Timestamp: 00:00:00,192 --> 00:00:05,222
   Selected Text: 'Welcome to this brand new course of the DTP. DTP के इस कोर्स में आप सभी का स्वागत है। मेरा नाम है जितेंद्र।'
----------------------------------------
Segment 2 - Timestamp: 00:00:05,782 --> 00:00:23,252
   Selected Text: 'मैं एक ग्राफिक मोशन ग्राफिक डिजाइनर हूं इस इंडस्ट्री में 15 प्लस ईयर से काम कर रहा हूं। यहां पर मैं आपको बताऊंगा चार इंपॉर्टेंट सॉफ्टवेयर्स जो मार्केट में चलते हैं एज अ डीटीपी डिजाइनर आपको पता होना चाहिए। जिसमें से पहला है Photoshop, फिर Illustrator, then InDesign और लास्ट में Corel Draw।'
----------------------------------------
Segment 3 - Timestamp: 00:00:23,662 --> 00:00:42,742
   Selected Text: 'ये कुछ ऐसे सॉफ्टवेयर्स हैं जिसका नॉलेज होने से इंडस्ट्री में जो रिक्वायरमेंट है उसको आप पूरा कर सकते हैं डिजाइनिंग

# test 1a sequential but for multi lang

In [4]:
from google.colab import drive, userdata
import os
import re
import time
from google import genai

# === Mount Google Drive and API ===
drive.mount('/content/drive')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

# === List of languages to process ===
# languages = [
#     "Hindi", "Tamil", "Telugu", "Bengali", "Kannada",
#     "Malayalam", "Marathi", "Gujarati", "Punjabi",
#     "Odia", "Assamese"
# ]

languages = [
     "Assamese"
]
# === Paths ===
base_dir = "/content/drive/My Drive/Test_28_Adnew_mp3/Test 1A ASRgemini/"
asr_dir = os.path.join(base_dir, "ASR_whisper1")     # input SRT files
mt_dir  = os.path.join(base_dir, "MT_gemini")        # master output folder
os.makedirs(mt_dir, exist_ok=True)

print("🟢 Starting Multi-Language Translation Pipeline...")

# === SRT Parsing Pattern ===
pattern = r"(\d+)\s+([\d:,]+ --> [\d:,]+)\s+(.+?)(?=\n\d+\n|$)"

def translate_batch(batch, target_language):
    """Translate a batch of subtitle lines."""
    joined_text = "\n".join([f"{num} {text}" for num, _, text in batch])

    prompt = f"""
You are a professional subtitle translator for Indic languages.

Translate the following subtitle dialog into {target_language}.
Preserve meaning. Keep subtitles natural and accurate.
Do NOT translate numbers or timestamps.
Return one line per subtitle, in order, with the segment number included.

Text:
{joined_text}
"""

    for _ in range(3):
        try:
            response = client.models.generate_content(
                model="gemini-2.5-pro",
                contents=prompt
            )
            return response.text.strip().split("\n")
        except Exception as e:
            print(f"   ⚠️ Error → Retrying batch... {e}")
            time.sleep(2)

    return [""] * len(batch)

# === Process ALL languages sequentially ===
for target_language in languages:

    print(f"\n🌐 === Translating into: {target_language} ===")

    # Create language-specific folder
    lang_dir = os.path.join(mt_dir, target_language)
    os.makedirs(lang_dir, exist_ok=True)
    print(f"📁 Output Folder → {lang_dir}")

    # Loop over SRT files
    for f_name in os.listdir(asr_dir):
        if not f_name.lower().endswith(".srt"):
            continue

        input_file = os.path.join(asr_dir, f_name)
        print(f"\n🎬 Processing: {f_name}")

        with open(input_file, "r", encoding="utf-8") as f:
            content = f.read()

        entries = re.findall(pattern, content, flags=re.DOTALL)
        print(f"   → {len(entries)} subtitle blocks detected")

        translated_entries = []
        translated_text_only = []

        batch_size = 10
        for i in range(0, len(entries), batch_size):
            batch = entries[i:i+batch_size]
            orig_texts = [(t[0], t[1], t[2].strip()) for t in batch]

            translated_batch = translate_batch(orig_texts, target_language)

            for (num, ts, _), trans in zip(orig_texts, translated_batch):
                trans = trans.replace(f"{num} ", "")
                translated_entries.append(f"{num}\n{ts}\n{trans}\n")
                translated_text_only.append(trans)

            print(f"   ✅ Translated segments {i+1}–{min(i+batch_size, len(entries))}")

        base = os.path.splitext(f_name)[0]
        srt_out = os.path.join(lang_dir, f"{base}_{target_language}.srt")
        txt_out = os.path.join(lang_dir, f"{base}_{target_language}.txt")

        with open(srt_out, "w", encoding="utf-8") as f:
            f.write("\n".join(translated_entries))

        with open(txt_out, "w", encoding="utf-8") as f:
            f.write("\n".join(translated_text_only))

        print(f"   📁 Saved SRT → {srt_out}")
        print(f"   📄 Saved TXT → {txt_out}")

print("\n🎉 ALL 11 LANGUAGES TRANSLATED SUCCESSFULLY!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🟢 Starting Multi-Language Translation Pipeline...

🌐 === Translating into: Assamese ===
📁 Output Folder → /content/drive/My Drive/Test_28_Adnew_mp3/Test 1A ASRgemini/MT_gemini/Assamese

🎬 Processing: Copy of Chapter 1A - Concept of Basic Electricity Voltage, Currents, Resistance, Impedance & Power Factor_Whisper1_eng_eng.srt
   → 304 subtitle blocks detected
   ✅ Translated segments 1–10
   ✅ Translated segments 11–20
   ✅ Translated segments 21–30
   ✅ Translated segments 31–40
   ✅ Translated segments 41–50
   ✅ Translated segments 51–60
   ⚠️ Error → Retrying batch... 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}
   ⚠️ Error → Retrying batch... 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}
 

# test mt 1a

In [1]:
from google.colab import drive, userdata
import os
import re
import time
from google import genai

# === Mount Google Drive and API ===
drive.mount('/content/drive')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

# === Paths ===
base_dir = "/content/drive/My Drive/Test_28_Adnew_mp3/Test 1A ASRgemini/"
asr_dir = os.path.join(base_dir, "ASR_whisper1")  # input SRTs
mt_dir = os.path.join(base_dir, "MT_gemini")  # translated output
srt_dir = os.path.join(mt_dir, "Srt")
txt_dir = os.path.join(mt_dir, "Txt")
os.makedirs(srt_dir, exist_ok=True)
os.makedirs(txt_dir, exist_ok=True)
os.makedirs(mt_dir, exist_ok=True)

target_language = "Hindi"
print("🟢 Ready — Processing all .srt files...")

# === SRT parsing pattern ===
pattern = r"(\d+)\s+([\d:,]+ --> [\d:,]+)\s+(.+?)(?=\n\d+\n|$)"

def translate_batch(batch):
    """Translate batch of subtitle text chunks with Gemini, using segment number."""
    # Prepare the joined text with segment number only (no timestamp)
    joined_text = "\n".join([f"{num} {text}" for num, _, text in batch])

    prompt = f"""
You are a professional subtitle translator for Indic languages.

Translate the following subtitle dialogue into {target_language}.
Preserve meaning. Keep subtitles natural.
Do NOT translate numbers or timestamps.
Return one line per subtitle, in order, with the segment number included.

Text:
{joined_text}
"""
    # Retry logic in case of error
    for _ in range(3):
        try:
            response = client.models.generate_content(
                model="gemini-2.5-pro",  # or gemini-2.0-pro if you have access
                contents=prompt
            )
            result_text = response.text.strip()
            # Split the response into lines
            return result_text.split("\n")
        except Exception as e:
            print("Retrying batch due to error:", e)
            time.sleep(3)

    # If all retries fail, return empty lines
    return [""] * len(batch)

# === Loop over all SRT files ===
for f_name in os.listdir(asr_dir):
    if not f_name.lower().endswith(".srt"):
        continue

    input_file = os.path.join(asr_dir, f_name)
    print(f"\n🎬 Processing: {f_name}")

    with open(input_file, 'r', encoding='utf-8') as f:
        content = f.read()

    entries = re.findall(pattern, content, flags=re.DOTALL)
    print(f"   → {len(entries)} subtitles detected")

    translated_entries = []
    translated_text_only = []

    batch_size = 10
    for i in range(0, len(entries), batch_size):
        batch = entries[i:i + batch_size]

        # Include segment number alongside the text
        orig_texts = [(t[0], t[1], t[2].strip()) for t in batch]  # Segment number, timestamp, and subtitle text

        # Send the batch to Gemini for translation
        translated_batch = translate_batch(orig_texts)

        # After translation, map back to original format (segment number, timestamp, and translated text)
        for (num, ts, _), trans in zip(orig_texts, translated_batch):
            # Remove the extra segment number from Gemini's response
            trans = trans.replace(f"{num} ", "")  # Remove the segment number from the translation
            translated_entries.append(f"{num}\n{ts}\n{trans}\n")
            translated_text_only.append(trans)

        print(f"   ✅ Translated segments {i+1}–{min(i + batch_size, len(entries))}")

    # Save outputs
    base = os.path.splitext(f_name)[0]
    srt_out = os.path.join(srt_dir, f"{base}_{target_language}.srt")
    txt_out = os.path.join(txt_dir, f"{base}_{target_language}.txt")

    with open(srt_out, "w", encoding='utf-8') as f:
        f.write("\n".join(translated_entries))

    with open(txt_out, "w", encoding='utf-8') as f:
        f.write("\n".join(translated_text_only))

    print(f"   📁 Saved → {srt_out}")
    print(f"   📄 Saved → {txt_out}")

print("\n✅ All files translated successfully!")


Mounted at /content/drive
🟢 Ready — Processing all .srt files...

🎬 Processing: Copy of Chapter 1A - Concept of Basic Electricity Voltage, Currents, Resistance, Impedance & Power Factor_Whisper1_eng_eng.srt
   → 304 subtitles detected
   ✅ Translated segments 1–10
   ✅ Translated segments 11–20
   ✅ Translated segments 21–30
   ✅ Translated segments 31–40
   ✅ Translated segments 41–50
   ✅ Translated segments 51–60
   ✅ Translated segments 61–70
   ✅ Translated segments 71–80
   ✅ Translated segments 81–90
   ✅ Translated segments 91–100
   ✅ Translated segments 101–110
   ✅ Translated segments 111–120
   ✅ Translated segments 121–130
   ✅ Translated segments 131–140
   ✅ Translated segments 141–150
   ✅ Translated segments 151–160
   ✅ Translated segments 161–170
   ✅ Translated segments 171–180
   ✅ Translated segments 181–190
   ✅ Translated segments 191–200
   ✅ Translated segments 201–210
   ✅ Translated segments 211–220
   ✅ Translated segments 221–230
   ✅ Translated segments 2

# good code

In [ ]:
from google.colab import drive, userdata
import os
import re
import time
from google import genai

# === Mount Google Drive and API ===
drive.mount('/content/drive')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

# === Paths ===
base_dir = "/content/drive/My Drive/Test_28_Adnew_wav/"
asr_dir = os.path.join(base_dir, "testgenmt","Alleng")  # input SRTs
mt_dir = os.path.join(base_dir, "testgenmt","Eng_others", "As")  # translated output
srt_dir = os.path.join(mt_dir, "Srt")
txt_dir = os.path.join(mt_dir, "Txt")
os.makedirs(srt_dir, exist_ok=True)
os.makedirs(txt_dir, exist_ok=True)
os.makedirs(mt_dir, exist_ok=True)

target_language = "Assamese"
print("🟢 Ready — Processing all .srt files...")

# === SRT parsing pattern ===
pattern = r"(\d+)\s+([\d:,]+ --> [\d:,]+)\s+(.+?)(?=\n\d+\n|$)"

def translate_batch(batch):
    """Translate batch of subtitle text chunks with Gemini, using segment number."""
    # Prepare the joined text with segment number only (no timestamp)
    joined_text = "\n".join([f"{num} {text}" for num, _, text in batch])

    prompt = f"""
You are a professional subtitle translator for Indic languages.

Translate the following subtitle dialogue into {target_language}.
Preserve meaning. Keep subtitles natural.
Do NOT translate numbers or timestamps.
Return one line per subtitle, in order, with the segment number included.

Text:
{joined_text}
"""
    # Retry logic in case of error
    for _ in range(3):
        try:
            response = client.models.generate_content(
                model="gemini-2.5-pro",  # or gemini-2.0-pro if you have access
                contents=prompt
            )
            result_text = response.text.strip()
            # Split the response into lines
            return result_text.split("\n")
        except Exception as e:
            print("Retrying batch due to error:", e)
            time.sleep(3)

    # If all retries fail, return empty lines
    return [""] * len(batch)

# === Loop over all SRT files ===
for f_name in os.listdir(asr_dir):
    if not f_name.lower().endswith(".srt"):
        continue

    input_file = os.path.join(asr_dir, f_name)
    print(f"\n🎬 Processing: {f_name}")

    with open(input_file, 'r', encoding='utf-8') as f:
        content = f.read()

    entries = re.findall(pattern, content, flags=re.DOTALL)
    print(f"   → {len(entries)} subtitles detected")

    translated_entries = []
    translated_text_only = []

    batch_size = 10
    for i in range(0, len(entries), batch_size):
        batch = entries[i:i + batch_size]

        # Include segment number alongside the text
        orig_texts = [(t[0], t[1], t[2].strip()) for t in batch]  # Segment number, timestamp, and subtitle text

        # Send the batch to Gemini for translation
        translated_batch = translate_batch(orig_texts)

        # After translation, map back to original format (segment number, timestamp, and translated text)
        for (num, ts, _), trans in zip(orig_texts, translated_batch):
            # Remove the extra segment number from Gemini's response
            trans = trans.replace(f"{num} ", "")  # Remove the segment number from the translation
            translated_entries.append(f"{num}\n{ts}\n{trans}\n")
            translated_text_only.append(trans)

        print(f"   ✅ Translated segments {i+1}–{min(i + batch_size, len(entries))}")

    # Save outputs
    base = os.path.splitext(f_name)[0]
    srt_out = os.path.join(srt_dir, f"{base}_{target_language}.srt")
    txt_out = os.path.join(txt_dir, f"{base}_{target_language}.txt")

    with open(srt_out, "w", encoding='utf-8') as f:
        f.write("\n".join(translated_entries))

    with open(txt_out, "w", encoding='utf-8') as f:
        f.write("\n".join(translated_text_only))

    print(f"   📁 Saved → {srt_out}")
    print(f"   📄 Saved → {txt_out}")

print("\n✅ All files translated successfully!")


Mounted at /content/drive
🟢 Ready — Processing all .srt files...

🎬 Processing: Chapter 10B - Packaging, Labeling & Branding_Gemini25pro_hi_hi.srt_English.srt
   → 98 subtitles detected
   ✅ Translated segments 1–10
   ✅ Translated segments 11–20
   ✅ Translated segments 21–30
   ✅ Translated segments 31–40
   ✅ Translated segments 41–50
   ✅ Translated segments 51–60
   ✅ Translated segments 61–70
   ✅ Translated segments 71–80
   ✅ Translated segments 81–90
   ✅ Translated segments 91–98
   📁 Saved → /content/drive/My Drive/Test_28_Adnew_wav/testgenmt/Eng_others/As/Srt/Chapter 10B - Packaging, Labeling & Branding_Gemini25pro_hi_hi.srt_English_Assamese.srt
   📄 Saved → /content/drive/My Drive/Test_28_Adnew_wav/testgenmt/Eng_others/As/Txt/Chapter 10B - Packaging, Labeling & Branding_Gemini25pro_hi_hi.srt_English_Assamese.txt

🎬 Processing: Chapter 3C - Soy Wax candle Preparation_Gemini25pro_hi_hi_English.srt
   → 96 subtitles detected
   ✅ Translated segments 1–10
   ✅ Translated segme

# alomst Good code

In [ ]:
from google.colab import drive, userdata
import os
import re
import time
from google import genai

# === Mount Google Drive and API ===
drive.mount('/content/drive')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

# === Paths ===
base_dir = "/content/drive/My Drive/Test1/"
asr_dir = os.path.join(base_dir, "Fixed_srt","goodfile")  # input SRTs
mt_dir = os.path.join(base_dir, "mt","Eng")  # translated output
os.makedirs(mt_dir, exist_ok=True)

target_language = "English"
print("🟢 Ready — Processing all .srt files...")

# === SRT parsing pattern ===
pattern = r"(\d+)\s+([\d:,]+ --> [\d:,]+)\s+(.+?)(?=\n\d+\n|$)"

def translate_batch_with_numbers(batch):
    """Translate batch of subtitle text chunks with Gemini, including segment numbers."""
    # Prepend segment number and timestamp to each subtitle line
    joined_text = "\n".join([f"{num} {ts} {text}" for num, ts, text in batch])

    prompt = f"""
You are a professional subtitle translator for Indic languages.

Translate the following subtitle dialogue into {target_language}.
Preserve meaning. Keep subtitles natural.
Do NOT translate numbers or timestamps.
Return one line per subtitle, in order. Include the segment number and timestamp in the output.

Text:
{joined_text}
"""
    # Retry logic in case of error
    for _ in range(3):
        try:
            response = client.models.generate_content(
                model="gemini-2.5-pro",  # or gemini-2.0-pro if you have access
                contents=prompt
            )
            result_text = response.text.strip()
            # Split the response into lines
            return result_text.split("\n")
        except Exception as e:
            print("Retrying batch due to error:", e)
            time.sleep(3)

    # If all retries fail, return empty lines
    return [""] * len(batch)

# === Loop over all SRT files ===
for f_name in os.listdir(asr_dir):
    if not f_name.lower().endswith(".srt"):
        continue

    input_file = os.path.join(asr_dir, f_name)
    print(f"\n🎬 Processing: {f_name}")

    with open(input_file, 'r', encoding='utf-8') as f:
        content = f.read()

    entries = re.findall(pattern, content, flags=re.DOTALL)
    print(f"   → {len(entries)} subtitles detected")

    translated_entries = []
    translated_text_only = []

    batch_size = 10
    for i in range(0, len(entries), batch_size):
        batch = entries[i:i + batch_size]

        # Include segment number and timestamp alongside the text
        orig_texts = [(t[0], t[1], t[2].strip()) for t in batch]  # Adding segment number and timestamp

        # Send the batch to Gemini for translation
        translated_batch = translate_batch_with_numbers(orig_texts)

        # After translation, map back to original format (segment number, timestamp, and translated text)
        for (num, ts, _), trans in zip(orig_texts, translated_batch):
            translated_entries.append(f"{num}\n{ts}\n{trans}\n")
            translated_text_only.append(trans)

        print(f"   ✅ Translated segments {i+1}–{min(i + batch_size, len(entries))}")

    # Save outputs
    base = os.path.splitext(f_name)[0]
    srt_out = os.path.join(mt_dir, f"{base}_{target_language}.srt")
    txt_out = os.path.join(mt_dir, f"{base}_{target_language}.txt")

    with open(srt_out, "w", encoding='utf-8') as f:
        f.write("\n".join(translated_entries))

    with open(txt_out, "w", encoding='utf-8') as f:
        f.write("\n".join(translated_text_only))

    print(f"   📁 Saved → {srt_out}")
    print(f"   📄 Saved → {txt_out}")

print("\n✅ All files translated successfully!")


Mounted at /content/drive
🟢 Ready — Processing all .srt files...

🎬 Processing: Copy of Chapter 1A - Introduction to DTP (2).srt
   → 46 subtitles detected
   ✅ Translated segments 1–10
   ✅ Translated segments 11–20
   ✅ Translated segments 21–30
   ✅ Translated segments 31–40
   ✅ Translated segments 41–46
   📁 Saved → /content/drive/My Drive/Test1/mt/Eng/Copy of Chapter 1A - Introduction to DTP (2)_English.srt
   📄 Saved → /content/drive/My Drive/Test1/mt/Eng/Copy of Chapter 1A - Introduction to DTP (2)_English.txt

✅ All files translated successfully!


# old

In [ ]:
from google.colab import drive, userdata
import os
import re
import time
from google import genai

# === Mount Google Drive and API ===
drive.mount('/content/drive')
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')
client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"])

# === Paths ===
base_dir = "/content/drive/My Drive/Test1/"
asr_dir = os.path.join(base_dir, "Fixed_srt","goodfile")  # input SRTs
mt_dir = os.path.join(base_dir, "mt","Eng")  # translated output
os.makedirs(mt_dir, exist_ok=True)

target_language = "English"
print("🟢 Ready — Processing all .srt files...")

# === SRT parsing pattern ===
pattern = r"(\d+)\s+([\d:,]+ --> [\d:,]+)\s+(.+?)(?=\n\d+\n|$)"

def translate_batch(lines):
    """Translate list of subtitle text chunks at once with Gemini."""
    joined_text = "\n".join(lines)
    prompt = f"""
You are a professional subtitle translator for Indic languages.

Translate the following subtitle dialogue into {target_language}.
Preserve meaning. Keep subtitles natural.
Do NOT translate numbers or timestamps.
Return same line per subtitle, in order.

Text:
{joined_text}
"""
    for _ in range(3):  # retry logic
        try:
            response = client.models.generate_content(
                model="gemini-2.5-pro",  # or gemini-2.0-pro if you have access
                contents=prompt
            )
            # Gemini's response object
            result_text = response.text.strip()
            return result_text.split("\n")
        except Exception as e:
            print("Retrying batch due to error:", e)
            time.sleep(3)
    return [""] * len(lines)

# === Loop over all SRT files ===
for f_name in os.listdir(asr_dir):
    if not f_name.lower().endswith(".srt"):
        continue

    input_file = os.path.join(asr_dir, f_name)
    print(f"\n🎬 Processing: {f_name}")

    with open(input_file, 'r', encoding='utf-8') as f:
        content = f.read()

    entries = re.findall(pattern, content, flags=re.DOTALL)
    print(f"   → {len(entries)} subtitles detected")

    translated_entries = []
    translated_text_only = []

    batch_size = 10
    for i in range(0, len(entries), batch_size):
        batch = entries[i:i+batch_size]
        orig_texts = [t[2].strip() for t in batch]

        translated_batch = translate_batch(orig_texts)

        for (num, ts, _), trans in zip(batch, translated_batch):
            translated_entries.append(f"{num}\n{ts}\n{trans}\n")
            translated_text_only.append(trans)

        print(f"   ✅ Translated segments {i+1}–{min(i+batch_size,len(entries))}")

    # Save outputs
    base = os.path.splitext(f_name)[0]
    srt_out = os.path.join(mt_dir, f"{base}_{target_language}.srt")
    txt_out = os.path.join(mt_dir, f"{base}_{target_language}.txt")

    with open(srt_out, "w", encoding='utf-8') as f:
        f.write("\n".join(translated_entries))

    with open(txt_out, "w", encoding='utf-8') as f:
        f.write("\n".join(translated_text_only))

    print(f"   📁 Saved → {srt_out}")
    print(f"   📄 Saved → {txt_out}")

print("\n✅ All files translated successfully!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🟢 Ready — Processing all .srt files...

🎬 Processing: Copy of Chapter 1A - Introduction to DTP (2).srt
   → 46 subtitles detected
   ✅ Translated segments 1–10
   ✅ Translated segments 11–20
   ✅ Translated segments 21–30
   ✅ Translated segments 31–40
   ✅ Translated segments 41–46
   📁 Saved → /content/drive/My Drive/Test1/mt/Eng/Copy of Chapter 1A - Introduction to DTP (2)_English.srt
   📄 Saved → /content/drive/My Drive/Test1/mt/Eng/Copy of Chapter 1A - Introduction to DTP (2)_English.txt

✅ All files translated successfully!


# audio size reduce under 25mb

In [ ]:
import os
from pydub import AudioSegment

# ========================
# HELPER FUNCTION TO REDUCE FILE SIZE
# ========================

def reduce_wav_size(file_path, target_size_mb=25, output_path=None):
    """
    Reduces the size of a .wav file by adjusting bitrate, sample rate, and channels.
    Ensures the resulting file is under the target size (in MB).
    """
    # Load the audio file
    audio = AudioSegment.from_wav(file_path)

    # Convert to mono (if stereo)
    audio = audio.set_channels(1)

    # Reduce sample rate (e.g., 22050 Hz instead of 44100 Hz)
    audio = audio.set_frame_rate(22050)

    # Export the reduced audio to a temporary location to check the size
    temp_path = output_path if output_path else file_path
    audio.export(temp_path, format="wav", codec="pcm_s16le")

    # Check the file size, and adjust further if necessary
    while os.path.getsize(temp_path) > target_size_mb * 1024 * 1024:
        print(f"File too large ({os.path.getsize(temp_path)} bytes). Reducing further...")

        # Reduce further sample rate, or trim silence
        audio = audio.set_frame_rate(audio.frame_rate - 1000)  # Reduce 1 kHz at a time

        # Re-export the file
        audio.export(temp_path, format="wav", codec="pcm_s16le")

    print(f"Final file size: {os.path.getsize(temp_path) / (1024 * 1024):.2f} MB")
    return temp_path

# ========================
# REDUCE FILE SIZE FOR ALL .WAV FILES IN THE INPUT FOLDER
# ========================

input_dir = "/content/drive/MyDrive/Test_28_Adnew_wav/English/a"  # Input folder path
output_dir = "/content/drive/MyDrive/Test_28_Adnew_wav/Reduced"  # Folder to save reduced files
os.makedirs(output_dir, exist_ok=True)

for filename in os.listdir(input_dir):
    if filename.lower().endswith(".wav"):
        file_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)

        print(f"🎧 Reducing size for {filename}...")

        # Reduce the file size
        reduced_file = reduce_wav_size(file_path, target_size_mb=25, output_path=output_path)

        print(f"✅ Reduced file saved at: {reduced_file}")
